# Testing the API

Satellite data starts being recored on {'date': '2015-06-13'}

In [2]:
import requests
import os
from IPython.display import Image

In [3]:
# All Data
url = 'https://epic.gsfc.nasa.gov/api/natural/all'

# Most Recent
url = 'https://epic.gsfc.nasa.gov/api/natural/'

# URL by date
date = '20180428'
url = f'https://epic.gsfc.nasa.gov/api/enhanced/date/{date}'
res = requests.get(url)

In [4]:
# Example data retrieved
print(f'There are {len(res.json())} images available')
res.json()[0]

There are 22 images available


{'identifier': '20180428005516',
 'caption': "This image was taken by NASA's EPIC camera onboard the NOAA DSCOVR spacecraft",
 'image': 'epic_RGB_20180428005516',
 'version': '02',
 'centroid_coordinates': {'lat': 21.489258, 'lon': 165.871582},
 'dscovr_j2000_position': {'x': 1082471.700001,
  'y': 765002.975361,
  'z': 519622.982812},
 'lunar_j2000_position': {'x': -371472.817493,
  'y': -85970.908502,
  'z': 29631.801617},
 'sun_j2000_position': {'x': 119549193.299457,
  'y': 84022262.300594,
  'z': 36423693.849281},
 'attitude_quaternions': {'q0': 0.1799,
  'q1': 0.04705,
  'q2': -0.936977,
  'q3': 0.295793},
 'date': '2018-04-28 00:50:27',
 'coords': {'centroid_coordinates': {'lat': 21.489258, 'lon': 165.871582},
  'dscovr_j2000_position': {'x': 1082471.700001,
   'y': 765002.975361,
   'z': 519622.982812},
  'lunar_j2000_position': {'x': -371472.817493,
   'y': -85970.908502,
   'z': 29631.801617},
  'sun_j2000_position': {'x': 119549193.299457,
   'y': 84022262.300594,
   'z': 36

In [14]:
# Available data time and coords
fetch = 3
points = []
for e in res.json()[:fetch]:
    points.append((e['image'], e['centroid_coordinates'], e['date']))
points

[('epic_RGB_20180428005516',
  {'lat': 21.489258, 'lon': 165.871582},
  '2018-04-28 00:50:27'),
 ('epic_RGB_20180428020044',
  {'lat': 21.489258, 'lon': 149.458008},
  '2018-04-28 01:55:55'),
 ('epic_RGB_20180428030612',
  {'lat': 21.533203, 'lon': 133.132324},
  '2018-04-28 03:01:23')]

In [10]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
img = f"https://epic.gsfc.nasa.gov/archive/{collection['enh']}/{date[:4]}/{date[4:6]}/{date[6:8]}/{size['med']}/{res.json()[0]['image']}.{size['med']}"
print(img)
display(Image(url=img))

https://epic.gsfc.nasa.gov/archive/enhanced/2018/04/28/jpg/epic_RGB_20180428005516.jpg


In [15]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
for e in res.json():
    img = f"https://epic.gsfc.nasa.gov/archive/{collection['enh']}/{date[:4]}/{date[4:6]}/{date[6:8]}/png/{e['image']}.{size['big']}"
    #display(Image(url=img))

In [6]:
# Example data retrieved
print(res.json()[0]['identifier'])
YYYYMMDD = res.json()[0]['identifier']
year = YYYYMMDD[:4]
month = YYYYMMDD[4:6]
day = YYYYMMDD[6:8]
print(year, month, day)

20170604003634
2017 06 04


# Basic getSat() by date

In [57]:
## VERSION 1
def getSat(img_type, YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    # Define types of collections
    collection = {'nat': 'natural', 'enh':'enhanced'}
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collection[img_type]}/date/{YYYY}-{MM}-{DD}"
    res = requests.get(metadata, params=queryParams)
    print(res.status_code, res.url)

    # Construct the resource url
    img = f"{url}/archive/{collection[img_type]}/{YYYY}/{MM}/{DD}/png/{e['image']}.{size['big']}"

    # Return json from requests' response body
    return res.json()

#getSat('nat', '2020', '04', '09')[1]

In [55]:
## VERSION 02
# Define when
def getSat(img_type, YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    # Define types of collections
    collection = {'nat': 'natural', 'enh':'enhanced'}
    date = f"{YYYY}-{MM}-{DD}"
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collection[img_type]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    # Construct the resource url
    #img = f"{url}/archive/{collection[img_type]}/{YYYY}/{MM}/{DD}/png/{e['image']}.{size['big']}"
    
    #Return relevant data as list
    return data

#getSat('nat', '2020', '03', '10')

In [64]:
## VERSION 03
# Define when
def getSat(collection, img_type,YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    date = f"{YYYY}-{MM}-{DD}"
    
    # Define types of image collections and size 
    collections = {'nat': 'natural', 'enh':'enhanced'}

    size = {'png': ['png', 'png'],
            'jpg': ['jpg', 'jpg'],
            'thumb': ['thumbs', 'jpg']}
    
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collections[collection]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    
    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    
    # Construct the resource url
    if len(data) > 0:
        for e in data:
            print(e)
            img = f"{host}/archive/{collections[collection]}/{YYYY}/{MM}/{DD}/{size[img_type][0]}/{e[0]}.{size[img_type][1]}"
            print(img)
    #Return relevant data as list
    return data

#getSat('nat', 'thumb', '2020', '03', '10')

In [74]:
## VERSION 04
def getSat(collection, img_type, YYYY,MM,DD, queryParams=dict()):
    
    date = f"{YYYY}-{MM}-{DD}"
    
    # Define types of image collections and size 
    collections = {'nat': 'natural', 'enh':'enhanced'}

    size = {'png': ['png', 'png'],
            'jpg': ['jpg', 'jpg'],
            'thumb': ['thumbs', 'jpg']}
    
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collections[collection]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    
    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    
    # Construct the image resource url
    satellite_images = []
    if len(data) > 0:
        for e in data:
            img = f"{host}/archive/{collections[collection]}/{YYYY}/{MM}/{DD}/{size[img_type][0]}/{e[0]}.{size[img_type][1]}"
            satellite_images.append([img, e[1]])
    #Return relevant data as list
    return satellite_images

images = getSat('enh', 'thumb', '2020', '03', '01')

200 https://epic.gsfc.nasa.gov/api/enhanced/date/2020-03-01
There are 12 satellite images available for this date: 2020-03-01


In [77]:
display(Image(url=images[0][0]))

# Working with datetime

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
df = pd.DataFrame('INPUT/volcanic-explosions.csv')

In [ ]:
date_rng = pd.date_range(start='1/1/2015', end='10/04/2020', freq='D')
date_rng

In [ ]:
df = pd.DataFrame(date_rng, columns=['date'])
df['data'] = np.random.randint(0,100, size=(len(date_rng)))
df.head()

In [ ]:
df['datetime'] = pd.to_datetime(df.date)
df = df.set_index('datetime')

#list only the second day of each month
#df[df.index.day == 2]

# calculate mean of each year
df.resample('M').mean()

In [ ]:
year= 2020
month= 2
day=6

datetime.date(year,month,day).weekday()

In [ ]:
datetime.date.today()